In [1]:
#import All Required Librarys
import os
import numpy as np
import pandas as pd 
import random
import cv2
from glob import glob
import matplotlib.pyplot as plt
%matplotlib inline


In [2]:
from keras.models import Model
from keras.layers import Flatten,Dense
from keras.applications.vgg16 import VGG16


In [3]:
IMAGESHAPE = [224, 224, 3] 
training_data = r'C:\Users\Asus\Desktop\Bronchitis Project\Other Datasets\chest_xray\test'
testing_data = r'C:\Users\Asus\Desktop\Bronchitis Project\Other Datasets\chest_xray\train'

In [4]:
vgg_model = VGG16(input_shape=IMAGESHAPE, weights='imagenet', include_top=False)

In [5]:
for each_layer in vgg_model.layers:
    each_layer.trainable = False

In [6]:
classes = glob(r'C:\Users\Asus\Desktop\Bronchitis Project\Other Datasets\chest_xray\train\*') 

In [7]:
flatten_layer = Flatten()(vgg_model.output)
prediction = Dense(len(classes), activation='softmax')(flatten_layer)

In [8]:
final_model = Model(inputs=vgg_model.input, outputs=prediction) 
final_model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     

In [9]:
final_model.compile( 
  loss='categorical_crossentropy',
  optimizer='adam',
  metrics=['accuracy']
)

In [10]:
from keras.preprocessing.image import ImageDataGenerator
train_datagen = ImageDataGenerator(rescale = 1./255, 
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)
testing_datagen = ImageDataGenerator(rescale =1./255)

In [11]:
training_set = train_datagen.flow_from_directory(r'C:\Users\Asus\Desktop\Bronchitis Project\Other Datasets\chest_xray\train', 
                                                 target_size = (224, 224),
                                                 batch_size = 4,
                                                 class_mode = 'categorical')

Found 5232 images belonging to 2 classes.


In [12]:
test_set = testing_datagen.flow_from_directory(r'C:\Users\Asus\Desktop\Bronchitis Project\Other Datasets\chest_xray\test',
                                               target_size = (224, 224),
                                               batch_size = 4,
                                               class_mode = 'categorical')

Found 624 images belonging to 2 classes.


In [13]:
fitted_model = final_model.fit( 
 training_set,
 validation_data=test_set,
 epochs=5,
 steps_per_epoch=len(training_set),
 validation_steps=len(test_set)
)

Epoch 1/5
1308/1308 [==============================] - 1419s 1s/step - loss: 0.2711 - accuracy: 0.9192 - val_loss: 0.3468 - val_accuracy: 0.9199
Epoch 2/5
1308/1308 [==============================] - 1618s 1s/step - loss: 0.2660 - accuracy: 0.9442 - val_loss: 0.5987 - val_accuracy: 0.9151
Epoch 3/5
1308/1308 [==============================] - 1399s 1s/step - loss: 0.2203 - accuracy: 0.9520 - val_loss: 0.4862 - val_accuracy: 0.9279
Epoch 4/5
1308/1308 [==============================] - 1714s 1s/step - loss: 0.2164 - accuracy: 0.9585 - val_loss: 0.5604 - val_accuracy: 0.9167
Epoch 5/5
1308/1308 [==============================] - 1578s 1s/step - loss: 0.2038 - accuracy: 0.9568 - val_loss: 1.0951 - val_accuracy: 0.8846


In [15]:
#save Final model
final_model.save(r'C:\Users\Asus\our_model.h5')

In [ ]:
# Code for testing 

In [17]:
from keras_preprocessing import image
from keras.models import load_model
from keras.applications.vgg16 import preprocess_input
import numpy as np
model=load_model(r'C:\Users\Asus\our_model.h5') #Loading our model
img=image.load_img(r"C:\Users\Asus\Desktop\Bronchitis Project\Other Datasets\chest_xray\train\PNEUMONIA\person9_bacteria_38.jpeg",target_size=(224,224))
imagee=image.img_to_array(img) #Converting the X-Ray into pixels
imagee=np.expand_dims(imagee, axis=0)
img_data=preprocess_input(imagee)
prediction=model.predict(img_data)
if prediction[0][0]>prediction[0][1]:  #Printing the prediction of model.
    print('Person is safe.')
else:
    print('Person is affected with Pneumonia.')
print(f'Predictions: {prediction}')

1/1 [==============================] - 1s 716ms/step
Person is affected with Pneumonia.
Predictions: [[0. 1.]]


In [ ]:
# full code 

from keras.models import Model
from keras.layers import Flatten,Dense
from keras.applications.vgg16 import VGG16  #Import all the necessary modules
import matplotlib.pyplot as plot
from glob import glob
  
IMAGESHAPE = [224, 224, 3] #Provide image size as 224 x 224 this is a fixed-size for VGG16 architecture
vgg_model = VGG16(input_shape=IMAGESHAPE, weights='imagenet', include_top=False)
#3 signifies that we are working with RGB type of images.
training_data = r'C:\Users\Asus\Desktop\Bronchitis Project\Other Datasets\chest_xray\test'
testing_data = r'C:\Users\Asus\Desktop\Bronchitis Project\Other Datasets\chest_xray\train' #Give our training and testing path
  
for each_layer in vgg_model.layers:
    each_layer.trainable = False #Set the trainable as False, So that all the layers would not be trained.
classes = glob(r'C:\Users\Asus\Desktop\Bronchitis Project\Other Datasets\chest_xray\train') #Finding how many classes present in our train dataset.
flatten_layer = Flatten()(vgg_model.output)
prediction = Dense(len(classes), activation='softmax')(flatten_layer)
final_model = Model(inputs=vgg_model.input, outputs=prediction) #Combine the VGG output and prediction , this all together will create a model.
final_model.summary() #Displaying the summary
final_model.compile( #Compiling our model using adam optimizer and optimization metric as accuracy.
  loss='categorical_crossentropy',
  optimizer='adam',
  metrics=['accuracy']
)
from keras.preprocessing.image import ImageDataGenerator
train_datagen = ImageDataGenerator(rescale = 1/255., #importing our dataset to keras using ImageDataGenerator in keras.
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)
testing_datagen = ImageDataGenerator(rescale =1/255.)
training_set = train_datagen.flow_from_directory(r'C:\Users\Asus\Desktop\Bronchitis Project\Other Datasets\chest_xray\test', #inserting the images.
                                                 target_size = (224, 224),
                                                 batch_size = 224,
                                                 class_mode = 'binary')
test_set = testing_datagen.flow_from_directory(r'C:\Users\Asus\Desktop\Bronchitis Project\Other Datasets\chest_xray\train',
                                               target_size = (224, 224),
                                               batch_size = 224,
                                               class_mode = 'binary')
#fitted model generator
fitted_model = final_model.fit_generator(
training_set,
validation_data=test_set,
epochs=10,
steps_per_epoch=len(training_set),
validation_steps=len(test_set)
)


final_model.save(r'C:\Users\Asus\our_model.h5') #Saving the model file.

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     

C:\Users\Asus\AppData\Local\Temp/ipykernel_15304/848059739.py:42: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  fitted_model = final_model.fit_generator(


In [ ]:
from keras.models import Model
from keras.layers import Flatten,Dense
from keras.applications.vgg16 import VGG16  #Import all the necessary modules
import matplotlib.pyplot as plot
from glob import glob
  
IMAGESHAPE = [224, 224, 3] #Provide image size as 224 x 224 this is a fixed-size for VGG16 architecture
vgg_model = VGG16(input_shape=IMAGESHAPE, weights='imagenet', include_top=False)
#3 signifies that we are working with RGB type of images.
training_data = r'C:\Users\Asus\Desktop\Bronchitis Project\Other Datasets\chest_xray\test'
testing_data = r'C:\Users\Asus\Desktop\Bronchitis Project\Other Datasets\chest_xray\train' #Give our training and testing path
  


In [ ]:
for each_layer in vgg_model.layers:
    each_layer.trainable = False #Set the trainable as False, So that all the layers would not be trained.
classes = glob(r'C:\Users\Asus\Desktop\Bronchitis Project\Other Datasets\chest_xray\train') #Finding how many classes present in our train dataset.
flatten_layer = Flatten()(vgg_model.output)
prediction = Dense(len(classes), activation='softmax')(flatten_layer)
final_model = Model(inputs=vgg_model.input, outputs=prediction) #Combine the VGG output and prediction , this all together will create a model.
final_model.summary() #Displaying the summary
final_model.compile( #Compiling our model using adam optimizer and optimization metric as accuracy.
  loss='categorical_crossentropy',
  optimizer='adam',
  metrics=['accuracy']
)

from keras.preprocessing.image import ImageDataGenerator
train_datagen = ImageDataGenerator(rescale = 1./255, #importing our dataset to keras using ImageDataGenerator in keras.
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)
testing_datagen = ImageDataGenerator(rescale =1. / 255)

In [ ]:

training_set = train_datagen.flow_from_directory(r'C:\Users\Asus\Desktop\Bronchitis Project\Other Datasets\chest_xray\test', #inserting the images.
                                                 target_size = (224, 224),
                                                 batch_size = 4,
                                                 class_mode = 'categorical')
test_set = testing_datagen.flow_from_directory(r'C:\Users\Asus\Desktop\Bronchitis Project\Other Datasets\chest_xray\train',
                                               target_size = (224, 224),
                                               batch_size = 4,
                                               class_mode = 'categorical')
 #Fitting the model.
fitted_model = final_model.fit(training_set,validation_data=test_set,epochs=10,steps_per_epoch=len(training_set),validation_steps=len(test_set)
)


In [ ]:
#check where the train model is save 

import os;
print(os.getcwd())

In [ ]:
pwd

In [ ]:
# testing code

from keras_preprocessing import image
from keras.models import load_model
from keras.applications.vgg16 import preprocess_input
import numpy as np
model=load_model(r'C:\Users\Asus\our_model.h5') #Loading our model
img=image.load_img(r"C:\Users\Asus\Desktop\NORMAL-1110860-0001.jpeg",target_size=(224,224))
imagee=image.img_to_array(img) #Converting the X-Ray into pixels
imagee=np.expand_dims(imagee, axis=0)
img_data=preprocess_input(imagee)
prediction=model.predict(img_data)
if prediction[0][0]>prediction[0][1]:  #Printing the prediction of model.
    print('Person is safe.')
else:
    print('Person is affected with Pneumonia.')
print(f'Predictions: {prediction}')

In [ ]:
pip install keras_preprocessing

In [ ]:
pip install tensorflow --upgrade

In [ ]:
import tensorflow as tf
print(tf.__version__)

In [ ]:
pwd

In [ ]:
pip install torch torchvision

In [ ]:
pip uninstall -y torch torchvision

In [ ]:
pip install tensorflow

In [ ]:
pip install tensorflow-gpu==1.14

In [ ]:
from tensorflow import keras


In [1]:
pip install keras-gpu

Note: you may need to restart the kernel to use updated packages.


ERROR: Could not find a version that satisfies the requirement keras-gpu (from versions: none)
ERROR: No matching distribution found for keras-gpu


In [2]:
pip install tensorflow-gpu

  Running setup.py clean for tensorflow-gpu
Failed to build tensorflow-gpu
    Running setup.py install for tensorflow-gpu: started
    Running setup.py install for tensorflow-gpu: finished with status 'error'
Note: you may need to restart the kernel to use updated packages.


  ERROR: Command errored out with exit status 1:
   command: 'D:\New folder\python.exe' -u -c 'import io, os, sys, setuptools, tokenize; sys.argv[0] = '"'"'C:\\Users\\Asus\\AppData\\Local\\Temp\\pip-install-fisoy45q\\tensorflow-gpu_2cb7b387464547e4898590dbfd76e624\\setup.py'"'"'; __file__='"'"'C:\\Users\\Asus\\AppData\\Local\\Temp\\pip-install-fisoy45q\\tensorflow-gpu_2cb7b387464547e4898590dbfd76e624\\setup.py'"'"';f = getattr(tokenize, '"'"'open'"'"', open)(__file__) if os.path.exists(__file__) else io.StringIO('"'"'from setuptools import setup; setup()'"'"');code = f.read().replace('"'"'\r\n'"'"', '"'"'\n'"'"');f.close();exec(compile(code, __file__, '"'"'exec'"'"'))' bdist_wheel -d 'C:\Users\Asus\AppData\Local\Temp\pip-wheel-gwz6yw38'
       cwd: C:\Users\Asus\AppData\Local\Temp\pip-install-fisoy45q\tensorflow-gpu_2cb7b387464547e4898590dbfd76e624\
  Complete output (17 lines):
  Traceback (most recent call last):
    File "<string>", line 1, in <module>
    File "C:\Users\Asus\AppDat

In [3]:
pip install keras-models

Note: you may need to restart the kernel to use updated packages.


ERROR: Exception:
Traceback (most recent call last):
  File "D:\New folder\lib\site-packages\pip\_vendor\urllib3\response.py", line 438, in _error_catcher
    yield
  File "D:\New folder\lib\site-packages\pip\_vendor\urllib3\response.py", line 519, in read
    data = self._fp.read(amt) if not fp_closed else b""
  File "D:\New folder\lib\site-packages\pip\_vendor\cachecontrol\filewrapper.py", line 62, in read
    data = self.__fp.read(amt)
  File "D:\New folder\lib\http\client.py", line 462, in read
    n = self.readinto(b)
  File "D:\New folder\lib\http\client.py", line 506, in readinto
    n = self.fp.readinto(b)
  File "D:\New folder\lib\socket.py", line 704, in readinto
    return self._sock.recv_into(b)
  File "D:\New folder\lib\ssl.py", line 1241, in recv_into
    return self.read(nbytes, buffer)
  File "D:\New folder\lib\ssl.py", line 1099, in read
    return self._sslobj.read(len, buffer)
socket.timeout: The read operation timed out

During handling of the above exception, anoth